In [258]:
%%capture
import os
import site
site.addsitedir('/home/schirrmr/.local/lib/python2.7/site-packages/')
site.addsitedir('/usr/lib/pymodules/python2.7/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/')
%cd /home/schirrmr/braindecode/code/braindecode/
assert 'THEANO_FLAGS' in os.environ
# switch to cpu
os.environ['THEANO_FLAGS'] = 'floatX=float32,device=cpu,nvcc.fastmath=True'
%load_ext autoreload
%autoreload 2
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

import matplotlib.lines as mlines
import seaborn
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
matplotlib.rcParams['font.size'] = 7
seaborn.set_style('darkgrid')

from braindecode.scripts.print_results import ResultPrinter
from braindecode.csp.print_results import CSPResultPrinter
from braindecode.analysis.pandas_util import (remove_indices_with_same_value,
                                              remove_columns_with_same_value,
                                              dataset_averaged_frame,
                                             load_data_frame,
                                             pairwise_compare_frame)
import logging
log = logging.getLogger()
log.setLevel("DEBUG")

## Basic Model

This is the network structure. In my multiple prediction implementation, when stride is used to half the dimension, the data is moved to the trial axis, not removed as normally would happen. So structure here does not show the whole network, but gives the basic idea.

I don't show batch normalization layers here.

|Layer/Block|#ResBlocks|Units|Filter Size|Nonlinearity/PoolingMode|Output Size|
|-|
|Input|-|22|-|-|1000x1|
|Conv|.|24|(22)x3x1|Relu/Elu|1000x1|
|ResBlocks|2/3|24|3x1|Relu/Elu|1000x1|
|ResBlock|1|48|3x1|Relu/Elu|500x1|
|ResBlocks|2/3|48|3x1|Relu/Elu|500x1|
|ResBlock|1|72|3x1|Relu/Elu|250x1|
|ResBlock|2/3|72|3x1|Relu/Elu|250x1|
|ResBlock|1|72|3x1|Relu/Elu|125x1|
|ResBlock|2/3|72|3x1|Relu/Elu|125x1|
|ResBlock|1|72|3x1|Relu/Elu|63x1|
|ResBlock|2/3|72|3x1|Relu/Elu|63x1|
|ResBlock|1|72|3x1|Relu/Elu|32x1|
|ResBlock|2/3|72|3x1|Relu/Elu|32x1|
|ResBlock|1|72|3x1|Relu/Elu|16x1|
|Pool|-|-|(8/10)x1|Mean|(~7)x1|
|Conv|-|-|1x1|Softmax|7x1|

Then afterwards reshape to get all predictions.

## BCI Competition

### Deep5

Note, Deep5 reaches around 71% on BCI Competition.


In [259]:
df_deep5_bcic = load_data_frame('data/models/paper/bci-competition/cnt/deep4/',
                            shorten_headers=True)
remove_indices_with_same_value(dataset_averaged_frame(df_deep5_bcic)[:2])[1:2]

time  \
                                                                    len   
layers f_time_len double_time_convs drop_prob f_len4 f_len3 f_len2        
cnt_4l 10         -                 -         10     10     10        9   

                                                                             \
                                                                      tmean   
layers f_time_len double_time_convs drop_prob f_len4 f_len3 f_len2            
cnt_4l 10         -                 -         10     10     10     00:18:05   

                                                                             \
                                                                       tstd   
layers f_time_len double_time_convs drop_prob f_len4 f_len3 f_len2            
cnt_4l 10         -                 -         10     10     10     00:05:32   

                                                                    test  \
                                                                    mean   
layers f_time_len double_time_convs drop_prob f_len4 f_len3 f_len2         
cnt_4l 10         -                 -         10     10     10      71.8   

                                                                          \
                                                                     std   
layers f_time_len double_time_convs drop_prob f_len4 f_len3 f_len2         
cnt_4l 10         -                 -         10     10     10      18.8   

                                                                   train       
                                                                    mean  std  
layers f_time_len double_time_convs drop_prob f_len4 f_len3 f_len2             
cnt_4l 10         -                 -         10     10     10      99.9  0.2

## ResNet

I tried out different variants. First an overview sorted by test mean accuracy.

* n_l_per_bl => Number of resblocks per residual "superblock" :)
* survival prob is for stocahstic depth. **1** are the same, indicates no stochastic depth.

#### Stochastid Depth+Elu Explanation

In the original paper (http://arxiv.org/abs/1603.09382), they do:

$$H_l = ReLU(b_l \ f_l(H_{l-1}) + H_{l-1})$$

Where $b_l$ is a bernoully random variable (either 0 or 1) at training time with probability $p_l$ to be 1.

At test time, $b_l$ is replaced with the probability $p_l$.

The idea here is that ReLU does not change $H_{l-1}$, since $H_{l-1}$ is also the output of a relu. So in case $b_l=0$, $H_l = H_{l-1}$ => ResBlock is bypassed.

We use ELU, and this is no longer true ($Elu(H_{l-1}) \ne H_{l-1}$). Therefore I decided to do it like this:


$$H_l =  b_l \cdot ELU (f_l(H_{l-1}) + id(H_{l-1})) + (1 - b_l) H_{l-1}$$

Again, during test time I replace $b_l$ with $p_l$.

#### Comparison our rule for ReLU

I am not sure this rule is a good idea since it would lead to same training behavior and different rest behavior for ReLU compared to theirs. 
We would have:

$$H_l =  b_l \cdot ReLU (f_l(H_{l-1}) + id(H_{l-1})) + (1 - b_l) H_{l-1}$$


##### Training

Training is identical, in case $b_l=0$, we get $H_l=H_{l-1}$, in case $b_l=1$, we get $H_l = ReLU(f_l(H_{l-1}) + H_{l-1})$. This is same as before.

##### Test

Test is different, example with $f_l(H_{l-1})=-3$ and $H_{l-1}=2$ and $p_l=0.5$.


* Theirs
$$H_l = ReLU(p_l \ f_l(H_{l-1}) + H_{l-1}) = ReLu(0.5\cdot -3 +2) = 0.5$$

* Ours

$$H_l = p_l  ReLU(f_l(H_{l-1}) + H_{l-1}) + (1 - p_l)H_{l-1} = 0.5\cdot ReLU( -3 +2) + 0.5 \cdot 2 = 0.5 ReLU(-1) + 1 = 1$$



In [297]:
data_frame = load_data_frame('data/models/paper/bci-competition/cnt/resnet/', params=dict(whisker_percent=None),
                            shorten_headers=True)
data_frame = data_frame[data_frame.nonlinearity != '-']
data_frame = data_frame.drop('final_nonlin', axis=1)
data_frame = data_frame.drop('batch_modifier', axis=1)

data_frame = data_frame.replace(dict(survival_prob={'-': 1}, final_agg={'-': 'pool'}, 
                                 drop_before_pool={'-': 'False'},
                                split_1st_layer={'-': False}))
averaged_frame = dataset_averaged_frame(data_frame)

In [298]:
df_avg = remove_indices_with_same_value(averaged_frame[averaged_frame['time', 'len'] > 1]).sort_values(
    by=[('test', 'mean')], ascending=False)
df_avg

time  \
                                                                                                                                                                                   len   
high_cut_hz bnorm_eps low_cut_off_hz final_p_len drop_before_pool cnt_preprocs                  n_1st_filters split_1st_layer survival_prob n_l_per_bl final_agg add_after_nonlin        
-           0.0001    null           10          True             resample_highpass_standardize 48            False           1.0           2          conv      -                   9   
                                                 False            resample_highpass_standardize 48            True            1.0           2          pool      -                   9   
                                                                                                64            False           1.0           2          pool      -                   9   
                                                 True             resample_highpass_standardize 64            False           1.0           2          conv      -                   9   
                                                                                                48            False           1.0           2          pool      -                   9   
                                                 False            resample_highpass_standardize 48            False           1.0           2          pool      -                   9   
                                                                                                64            False           1.0           2          conv      -                   9   
                                                                                                48            False           1.0           1          pool      -                   9   
                                                 True             resample_highpass_standardize 64            False           1.0           2          pool      -                   9   
                                                 False            resample_highpass_standardize 48            False           1.0           2          conv      -                   9   
                                                                                                24            False           1.0           2          pool      -                   9   
            0.0100    null           10          False            resample_highpass_standardize 48            False           1.0           2          pool      -                   9   
            0.0001    null           10          False            resample_highpass_standardize 48            False           1.0           2          pool      True                9   
                                                 True             resample_highpass_standardize 24            False           1.0           2          pool      -                   9   
            0.0100    null           10          True             resample_highpass_standardize 48            False           1.0           2          pool      -                   9   
            0.0001    null           10          False            resample_highpass_standardize 24            False           1.0           2          conv      -                   9   
                                                                                                48            False           0.5           2          pool      -                   9   
                                                 True             resample_highpass_standardize 24            False           1.0           2          conv      -                   9   
                                     9           False            resample_highpass_standardize 24            False           1.0           2          pool      -                   9   
                                     10          False            resample_highpass_st

In [288]:
df_after_nonlin = df_avg.xs(True, level='add_after_nonlin')

In [290]:
df_after_nonlin

time  \
                                                                                                                                                                  len   
high_cut_hz bnorm_eps low_cut_off_hz final_p_len drop_before_pool cnt_preprocs                  n_1st_filters split_1st_layer survival_prob n_l_per_bl final_agg        
-           0.0001    null           10          False            resample_highpass_standardize 48            False           1             2          pool         9   
                      4              10          False            resample_highpass_standardize 48            False           1             2          pool         9   
4           0.0001    -              10          False            resample_lowpass_standardize  48            False           1             2          pool         9   

                                                                                                                                                                           \
                                                                                                                                                                    tmean   
high_cut_hz bnorm_eps low_cut_off_hz final_p_len drop_before_pool cnt_preprocs                  n_1st_filters split_1st_layer survival_prob n_l_per_bl final_agg            
-           0.0001    null           10          False            resample_highpass_standardize 48            False           1             2          pool      00:41:16   
                      4              10          False            resample_highpass_standardize 48            False           1             2          pool      00:43:16   
4           0.0001    -              10          False            resample_lowpass_standardize  48            False           1             2          pool      00:36:39   

                                                                                                                                                                           \
                                                                                                                                                                     tstd   
high_cut_hz bnorm_eps low_cut_off_hz final_p_len drop_before_pool cnt_preprocs                  n_1st_filters split_1st_layer survival_prob n_l_per_bl final_agg            
-           0.0001    null           10          False            resample_highpass_standardize 48            False           1             2          pool      00:10:12   
                      4              10          False            resample_highpass_standardize 48            False           1             2          pool      00:10:48   
4           0.0001    -              10          False            resample_lowpass_standardize  48            False           1             2          pool      00:08:09   

                                                                                                                                                                  test  \
                                                                                                                                                                  mean   
high_cut_hz bnorm_eps low_cut_off_hz final_p_len drop_before_pool cnt_preprocs                  n_1st_filters split_1st_layer survival_prob n_l_per_bl final_agg         
-           0.0001    null           10          False            resample_highpass_standardize 48            False           1             2          pool       62.3   
                      4              10          False            resample_highpass_standardize 48            False           1             2          pool       57.3   
4           0.0001    -              10          False            resample_lowpass_standardize  48            False           1             2          pool       46.0   

                                               

In [299]:
dfc = df_avg.xs(0.0001, level='bnorm_eps')
dfc = dfc.xs(False, level='drop_before_pool')
dfc = dfc.xs('pool', level='final_agg')
dfc = dfc.xs(False, level='split_1st_layer')
dfc

time  \
                                                                                                                              len   
high_cut_hz low_cut_off_hz final_p_len cnt_preprocs                  n_1st_filters survival_prob n_l_per_bl add_after_nonlin        
-           null           10          resample_highpass_standardize 64            1.0           2          -                   9   
                                                                     48            1.0           2          -                   9   
                                                                                                 1          -                   9   
                                                                                                 2          True                9   
                                                                                   0.5           2          -                   9   
            4              10          resample_highpass_standardize 48            1.0           2          True                9   
4           -              10          resample_lowpass_standardize  48            1.0           2          True                9   

                                                                                                                                       \
                                                                                                                                tmean   
high_cut_hz low_cut_off_hz final_p_len cnt_preprocs                  n_1st_filters survival_prob n_l_per_bl add_after_nonlin            
-           null           10          resample_highpass_standardize 64            1.0           2          -                01:49:08   
                                                                     48            1.0           2          -                01:27:46   
                                                                                                 1          -                00:24:30   
                                                                                                 2          True             00:41:16   
                                                                                   0.5           2          -                00:49:05   
            4              10          resample_highpass_standardize 48            1.0           2          True             00:43:16   
4           -              10          resample_lowpass_standardize  48            1.0           2          True             00:36:39   

                                                                                                                                       \
                                                                                                                                 tstd   
high_cut_hz low_cut_off_hz final_p_len cnt_preprocs                  n_1st_filters survival_prob n_l_per_bl add_after_nonlin            
-           null           10          resample_highpass_standardize 64            1.0           2          -                00:34:36   
                                                                     48            1.0           2          -                00:28:10   
                                                                                                 1          -                00:06:50   
                                                                                                 2          True             00:10:12   
                                                                                   0.5           2          -                00:06:20   
            4              10          resample_highpass_standardize 48            1.0           2          True             00:10:48   
4           -              10          resample_lowpass_standardize  48            1.0           2          True             00:08:09   

                                                    

### Individual parameters

Here I compare individual parameters when remaining parameters are held fixed.
Note that mean accuracy values can now come from any combination of datasets, i.e. also from just a single dataset and should not be compared to accuracy values over all datasets (from table above).

In [300]:
compare_frame = pairwise_compare_frame(data_frame, with_p_vals=True)

compare_frame.sort_values(by='perm')

,n_exp,val_1,val_2,acc_1,acc_2,diff,std,perm
n_1st_filters,27,48,24,64.6,62.1,-2.5,4.3,0.7
n_l_per_bl,18,2,3,68.4,64.9,-3.5,5.2,0.9
n_1st_filters,27,64,24,64.3,62.1,-2.1,4.5,2.3
nonlinearity,4,elu,relu,79.2,67.0,-12.2,6.3,12.5
final_p_len,9,9,8,61.6,59.1,-2.5,4.3,15.6
final_p_len,13,10,8,62.8,60.6,-2.2,5.0,15.7
final_p_len,9,10,9,63.2,61.6,-1.6,3.1,16.4
low_cut_off_hz,9,null,4,62.3,57.3,-5.1,10.1,19.5
bnorm_eps,18,0.0001,0.01,64.2,62.6,-1.6,5.0,20.9
1st_f_len,4,3,11,66.3,59.7,-6.6,6.1,25.0


## Our Data

### Deep5

Note, deep5 reaches around 90.6% atm

In [263]:
df_deep5_ours = load_data_frame('data/models/paper/ours/cnt/deep4/', params=dict(whisker_percent=None),
                            shorten_headers=True)
dataset_averaged_frame(df_deep5_ours).sort_values(by=('test', 'mean'))[1:2]

ValueError: Grouper for 'cleaner' not 1-dimensional

## ResNet

In [307]:
data_frame = load_data_frame('data/models/paper/ours/cnt/resnet/', params=dict(whisker_percent=None),
                            shorten_headers=True)
data_frame = data_frame.drop('final_nonlin', axis=1)
data_frame = data_frame.replace(dict(survival_prob={'-': 1}, final_agg={'-': 'pool'},
                                split_1st_layer={'-': False}))
#data_frame = data_frame.drop('batch_modifier', axis=1)


In [308]:
remove_columns_with_same_value(dataset_averaged_frame(data_frame).sort_values(by=('test', 'mean'),
                                               ascending=False))

time  \
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                tmean   
marker_def      projection iterator     test=valid loss_expression                batch_norm_alpha dataset              l2_factor trial_stop max_epochs layer_names_to_norms bnorm_eps updates_expression low_cut_off_hz loader_type final_p_len run_after_early_stop resample_fs monitors              >epochs save_path                          high_cut_hz drop_before_pool load_sensor_names early_stop_chan cleaner    layers       trial_start test_cleaner 1st_f_len sensor_names cnt_preprocs                  #batch splitter              split_1st_layer preproc num_cv_folds survival_prob input_len nonlinearity n_l_per_bl preds n_1st_filters final_agg batch_modifier valid_set_fraction add_after_nonlin updates_modifier             
markers_4_class False      cnt_iterator False      safe_tied_neighbours_decay     0.1              cleaned_combined_set 0.0001    4000       500        layer_names_to_norms 0.0001    adam               null           BBCIDataset 10          True                 250         sample_trial_monitors 80      data/models/paper/ours/cnt/resnet/ -           False            null              valid_misclass  no_cleaner residual_net 1500        no_cleaner   3         C_sensors    resample_highpass_standardize 60     several_sets_splitter True            null    5            1.0           1000      elu          2          500   48            pool      null           0.2                -                max_norm_modifier 04:53:09   
                                                   categorical_crossentropy_decay 0.1              cleaned_combined_set 0.0001    4000       500        layer_names_to_norms 0.0001    adam               null           BBCIDataset 10          True                 250         sample_trial_monitors 80      data/models/paper/ours/cnt/resnet/ -           False            null              valid_misclass  no_cleaner residual_net 1500        no_cleaner   3         C_sensors    resample_highpass_standardize 60     several_sets_splitter False           null    5            1.0           1000      elu          2          500   48            conv      null           0.2                -                max_norm_modifier 02:23:39   
                                                   safe_tied_neighbours_decay     0.1              cleaned_combined_set 0.0001    4000       500        layer_names_to_norms 0.0001    adam               null           BBCIDataset 10          True                 250         sample_trial_monitors 80      data/models/paper/ours/cnt/resnet/ -           False            null              valid_misclass  no_cleaner residual_net 1500        no_cleaner   3         C_sensors    resample_highpass_standardize 60     several_sets_splitter False           null    5            1.0           1000      elu          2          500   48            pool      null           0.2                -                max_norm_modifier 02:04:45   
                                                   categorical_crossentropy_decay 0.1              cleaned_combined_set 0.0001    4000       500        layer_names_to_norms 0.0001    adam               null           BBCIDataset 10          True                 250         sample_trial_monitors 80      data/models/paper/ours/cnt/resnet/ -       

#### Individual parameters

In [305]:
compare_frame = pairwise_compare_frame(data_frame, with_p_vals=True)
compare_frame.sort_values(by='perm')

,n_exp,val_1,val_2,acc_1,acc_2,diff,std,perm
survival_prob,20,1.0,0.5,87.5,82.8,-4.7,3.1,0.0
add_after_nonlin,20,-,True,87.5,85.9,-1.7,3.3,5.0
n_l_per_bl,20,2,1,87.5,86.8,-0.8,2.3,20.4
loss_expression,20,safe_tied_neighbours_decay,categorical_crossentropy_decay,87.5,86.8,-0.7,2.7,31.2
low_cut_off_hz,20,null,4,85.9,83.9,-2.0,9.0,35.4
split_1st_layer,20,True,False,88.6,87.5,-1.1,5.3,48.1
drop_before_pool,40,False,True,87.2,86.8,-0.4,3.3,48.9
final_agg,40,conv,pool,87.2,86.9,-0.2,3.3,68.5
